# Project 03 - Due Monday, November 13 at 12pm

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.

*Attestation*: **Work in groups**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Monday, November 13, 12pm.  One member of each group must submit your notebook to Blackboard; you should not submit it separately.

# Group Member: Leiwen Lin, Huiting Tang, Xiaoli Zhao, Ziyi Zhao

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered using GWU Libraries' [Social Feed Manager](http://sfm.library.gwu.edu/) application during a recent game of the MLB World Series featuring the Los Angeles Dodgers and Houston Astros.  This first file tells you a little bit about how it was gathered:

In [1]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt

--2017-11-12 00:27:31--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920 (1.9K) [text/plain]
Saving to: ‘9670f3399f774789b7c3e18975d25611-README.txt.1’

9670f3399f774789b7c 100%[===================>]   1.88K  --.-KB/s    in 0s      

2017-11-12 00:27:31 (148 MB/s) - ‘9670f3399f774789b7c3e18975d25611-README.txt.1’ saved [1920/1920]



In [2]:
!cat 9670f3399f774789b7c3e18975d25611-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: 9670f3399f774789b7c3e18975d25611
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2017, 11:21:04 p.m. EDT
Deduplicate: No

COLLECTION INFORMATION
Collection name: test set for world series
Collection id: 34e3f7460b5c4df09d64a1e61fd81238
Collection set: mlb-test (collection set id d6e8c27b1bc942e78790aa55a82b3a7a)
Harvest type: Twitter filter
Collection description: running for just one hour, just for fun.

Harvest options:
Media: No
Web resources: No

Seeds:
* Track: dodgers,astros - Active

Change log:

Change to test set for world series (collection) on Oct. 30, 2017, 10:59:56 p.m. EDT by dchud:
* is_active: "True" changed to "False"

Change to test set for world series (collection) on Oct. 30, 2017, 10:58:51 p.m. EDT by dchud:
* is_on: "True" changed to "False"

Change to test set for world series (collection) on Oct. 29, 2017, 8:01:24 p.m. EDT by dch

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "astros".  Every item in this set should refer to one or the other, or both.
 * This data was not deduplicated; we may see individual items more than once.
 * Data was collected between October 29 and October 30.  Game 5 of the Series was played during this time.
 
You should not need to know anything about baseball to complete this assignment.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
 
### Q1.1 - Select at least one and obtain it using `wget`.  Verify the file sizes using the command line.

Each file should contain exactly 100,000 tweets.  

*Note*: you are only required to use one of these files, but you may use more than one.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

**We chose the first data set.**

In [3]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json

--2017-11-09 02:57:18--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.22.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595711407 (568M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_003.json’

9670f3399f774789b7c 100%[===================>] 568.11M  77.0MB/s    in 7.5s    

2017-11-09 02:57:26 (75.7 MB/s) - ‘9670f3399f774789b7c3e18975d25611_003.json’ saved [595711407/595711407]



**Verifying the file size**

In [3]:
!ls -lh 9670f3399f774789b7c3e18975d25611_003.json

-rw-rw-r-- 1 ubuntu ubuntu 569M Nov  3 11:25 9670f3399f774789b7c3e18975d25611_003.json


For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [4]:
!cat *.json | shuf -n 1 > example-tweet.json

In [5]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "entities": {
    "media": [
      {
        "id": 924916209103130625,
        "display_url": "pic.twitter.com/7NACbpBqDh",
        "media_url_https": "https://pbs.twimg.com/media/DNX2ZEqX0AElPkK.jpg",
        "indices": [
          87,
          110
        ],
        "media_url": "http://pbs.twimg.com/media/DNX2ZEqX0AElPkK.jpg",
        "expanded_url": "https://twitter.com/yemekyapmaknet/status/924916211514830848/photo/1",
        "id_str": "924916209103130625",
        "type": "photo",
        "url": "https://t.co/7NACbpBqDh",
        "sizes": {
          "thumb": {
            "w": 150,
            "resize": "crop",
            "h": 150
          },
          "large": {
            "w": 1050,
            "resize": "fit",
            "h": 549
          },
          "medium": {
            "w": 1050,
            "resize": "fit",
            "h": 549
          },
          "small": {
            "w": 680,
            "resize": "fit",
            "h": 356
          }
        }
    

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

### Q1.2 - Start up Spark, and verify the file sizes.

We will use our normal startup sequence here:

In [1]:
import os

In [2]:
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
from pyspark import SparkContext

In [6]:
spark = SparkContext(appName='project-03')

In [7]:
spark

<SparkContext master=local[*] appName=project-03>

In [8]:
from pyspark import SQLContext

In [9]:
sqlc = SQLContext(spark)

In [10]:
sqlc

Input tweet data set to spark.

In [11]:
tweets = sqlc.read.json("9670f3399f774789b7c3e18975d25611_*.json")

Verify that Spark has loaded the same number of tweets you saw before:

**Answer**

**There is 100,000 records. Thus, we can verify that Spark has loaded the same number.**

In [24]:
tweets.count()

100000

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

In [16]:
tweets

DataFrame[contributors: string, coordinates: struct<coordinates:array<double>,type:string>, created_at: string, display_text_range: array<bigint>, entities: struct<hashtags:array<struct<indices:array<bigint>,text:string>>,media:array<struct<display_url:string,expanded_url:string,id:bigint,id_str:string,indices:array<bigint>,media_url:string,media_url_https:string,sizes:struct<large:struct<h:bigint,resize:string,w:bigint>,medium:struct<h:bigint,resize:string,w:bigint>,small:struct<h:bigint,resize:string,w:bigint>,thumb:struct<h:bigint,resize:string,w:bigint>>,source_status_id:bigint,source_status_id_str:string,source_user_id:bigint,source_user_id_str:string,type:string,url:string>>,symbols:array<struct<indices:array<bigint>,text:string>>,urls:array<struct<display_url:string,expanded_url:string,indices:array<bigint>,url:string>>,user_mentions:array<struct<id:bigint,id_str:string,indices:array<bigint>,name:string,screen_name:string>>>, extended_entities: struct<media:array<struct<display_

To issue SQL queries, we need to register a table based on `tweets`:

In [25]:
tweets.createOrReplaceTempView("tweets")

### Q2.1 - Which 10 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL.

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

In [14]:
tweets.select("lang").groupBy("lang").count().orderBy("count",ascending=False).show(10)

+----+-----+
|lang|count|
+----+-----+
|  en|88987|
|  es| 6825|
| und| 3058|
|  in|  210|
|  fr|  181|
|  pt|  133|
|  nl|   89|
|  ht|   83|
|  ja|   81|
|  tl|   77|
+----+-----+
only showing top 10 rows



**The 10 most commonly used languages are English, Spanish, undertermined language, Indonesian, french, Portuguese, Dutch, Haitian Creole,Japanese, and Tagalog.**

In [20]:
sqlc.sql("""
SELECT lang AS language, COUNT(*) AS count
FROM tweets
GROUP BY language
ORDER BY count DESC
LIMIT 10
""").show()

+--------+-----+
|language|count|
+--------+-----+
|      en|88987|
|      es| 6825|
|     und| 3058|
|      in|  210|
|      fr|  181|
|      pt|  133|
|      nl|   89|
|      ht|   83|
|      ja|   81|
|      tl|   77|
+--------+-----+



**We got the same result by SQL query and spark python code.**

### Q2.2 - Which 10 time zones are most common among users?  Verify your result with both the dataframe and SQL.

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

In [21]:
tweets.select('user.time_zone').groupBy('time_zone').count().orderBy('count',ascending=False).show(10)

+--------------------+-----+
|           time_zone|count|
+--------------------+-----+
|                null|42239|
|Central Time (US ...|17436|
|Pacific Time (US ...|17097|
|Eastern Time (US ...| 8664|
|             Arizona| 2490|
|Mountain Time (US...| 2475|
|Atlantic Time (Ca...| 1048|
|             Caracas| 1006|
|              Hawaii|  821|
|         Mexico City|  793|
+--------------------+-----+
only showing top 10 rows



In [22]:
sqlc.sql("""
SELECT user.time_zone AS time_zone, COUNT(*) AS count
FROM tweets
GROUP BY time_zone
ORDER BY count DESC
LIMIT 10
""").show()

+--------------------+-----+
|           time_zone|count|
+--------------------+-----+
|                null|42239|
|Central Time (US ...|17436|
|Pacific Time (US ...|17097|
|Eastern Time (US ...| 8664|
|             Arizona| 2490|
|Mountain Time (US...| 2475|
|Atlantic Time (Ca...| 1048|
|             Caracas| 1006|
|              Hawaii|  821|
|         Mexico City|  793|
+--------------------+-----+



**We got same result by SQL query and spark python code.**

### Q2.3 - How many tweets mention the Dodgers?  How many mention the Astros?  How many mention both?

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

**We assumed "Dodgers" and "Astros" should be capital insensitive.**

**We transfromed all text content to lower case to achieve capital insensitive search.**

**The reason why we used SQL to answer this question was that we believed the SQL syntax "like" could make our queries clearly  readable.**

In [50]:
sqlc.sql("""
SELECT COUNT(*) AS count 
FROM tweets
WHERE Lower(tweets.text) LIKE '%dodgers%'
""").show()

+-----+
|count|
+-----+
|33114|
+-----+



In [51]:
sqlc.sql("""
SELECT COUNT(*) AS count FROM tweets
WHERE Lower(text) LIKE '%astros%' 
""").show()

+-----+
|count|
+-----+
|70149|
+-----+



In [56]:
sqlc.sql("""
SELECT COUNT(*) AS count FROM tweets
WHERE lower(text) LIKE '%astros%' AND lower(text) LIKE '%dodgers%'
""").show()

+-----+
|count|
+-----+
|13962|
+-----+



## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location

In which users' locations are the Astros and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings.

Hint:  you may use either the time zones or user-specified locations for this question.

**Answer**

**In the following answers, we didn't clean null value since we believed understanding what data was present and what was missing was also important.**

In [59]:
sqlc.sql("""
SELECT user.location, COUNT(*) AS count
FROM tweets
WHERE lower(text) LIKE '%dodgers%'
GROUP BY location
ORDER BY count DESC
""").show()

+-------------------+-----+
|           location|count|
+-------------------+-----+
|               null|10064|
|    Los Angeles, CA| 1192|
|        Houston, TX|  721|
|    California, USA|  565|
|        Los Angeles|  451|
|      United States|  379|
|         Texas, USA|  226|
|          Venezuela|  203|
|         California|  171|
|             México|  136|
|      Las Vegas, NV|  107|
|        Chicago, IL|  104|
|              Texas|  102|
|    San Antonio, TX|  100|
|            Houston|   94|
|                USA|   87|
|      San Diego, CA|   81|
|                 LA|   78|
|Southern California|   76|
|     Long Beach, CA|   75|
+-------------------+-----+
only showing top 20 rows



**It is pretty reasonable that users in Los Angeles mentioned Dodgers most excluding null value because Dodgers is a team from Los Angeles.**

In [61]:
sqlc.sql("""
SELECT user.location, COUNT(*) AS count
FROM tweets
WHERE lower(text) LIKE '%astros%'
GROUP BY location
ORDER BY count DESC
""").show()

+-------------------+-----+
|           location|count|
+-------------------+-----+
|               null|19352|
|        Houston, TX| 5670|
|         Texas, USA| 1213|
|              Texas|  738|
|            Houston|  696|
|    Los Angeles, CA|  679|
|     Houston, Texas|  564|
|      United States|  528|
|         Austin, TX|  406|
|    California, USA|  394|
|    San Antonio, TX|  381|
|                HTX|  277|
|        Houston, Tx|  269|
|        Los Angeles|  235|
|College Station, TX|  212|
|        Chicago, IL|  204|
|         Dallas, TX|  200|
|             México|  179|
|          Venezuela|  162|
|                htx|  162|
+-------------------+-----+
only showing top 20 rows



**It is pretty reasonable that users in Houston mentioned Astros most excluding null value because Astros is a team from Houston.**

### Q3.2 - Which Twitter users are being replied to the most?

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

In [73]:
sqlc.sql("""
SELECT in_reply_to_screen_name AS name, COUNT(*) AS count
FROM tweets
WHERE in_reply_to_screen_name IS NOT NULL
GROUP BY name
ORDER BY count DESC
""").show()

+---------------+-----+
|           name|count|
+---------------+-----+
|         astros|  821|
|        Dodgers|  624|
|            MLB|  196|
|  stephenasmith|  106|
|       MLBONFOX|   68|
|  DodgerInsider|   61|
|  Nick_Offerman|   54|
|        ABREG_1|   49|
|        trvisXX|   46|
|JustinVerlander|   42|
|   katiedtellez|   36|
|TitanicBaseball|   30|
|    TeddyCatnip|   24|
|      FOXSports|   22|
|      benmaller|   22|
|  rolandsmartin|   22|
|     JoelEmbiid|   21|
| DonnieWahlberg|   21|
|     LADominate|   21|
|  BlainVincent7|   20|
+---------------+-----+
only showing top 20 rows



**ANS: "astros" was the Twitter user who had the most replies.**

### Q3.3 - Which 10 verified users have the most followers?  Which 10 unverified users have the most followers?

Provide both the screen names and follower counts for each.

Discuss your findings.

**Answer**

In [18]:
sqlc.sql("""
SELECT user.screen_name, max(user.followers_count) AS counts
FROM tweets
WHERE user.verified = 'true'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").show()

+--------------+--------+
|   screen_name|  counts|
+--------------+--------+
|       Reuters|18937529|
|       FoxNews|16272836|
|           ABC|12551437|
|washingtonpost|11417638|
|           MLB| 7841255|
|           NPR| 7289492|
|   BillSimmons| 6000106|
|       NBCNews| 5442705|
|    JohnLegere| 4630104|
|     ANCALERTS| 4453229|
+--------------+--------+



**The top 10 verified users who had the most followers are showed above.**

In [19]:
sqlc.sql("""
SELECT user.screen_name, max(user.followers_count) AS counts
FROM tweets
WHERE user.verified = 'false'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").show()

+---------------+------+
|    screen_name|counts|
+---------------+------+
|        chochos|833669|
|  el_carabobeno|725952|
|       PAMsLOvE|712254|
|        jilevin|568341|
|    sun_das_ill|559669|
|       EP_Mundo|538525|
|         LALATE|516139|
|  piercearrow33|503015|
|      BigNeechi|496825|
|periodicovzlano|493446|
+---------------+------+



**The top 10 unverified users who had the most followers are showed above.**

### Q3.4 - What are the most popular sets of hashtags among users with many followers?  Are they the same as among users with few followers?

Decide for yourself exactly how many followers you believe to be "many", and explain your decision.  You may use queries and statistics to support this decision if you wish.

Hint: if your sample tweet above does not include hashtags under the `entities` field, generate a new example by running the `shuf` command again until you find one that does.

Hint 2: the hashtag texts will be in an array, so you may need some functions you haven't used before.  If you're using SQL, see the docs for [Hive SQL](https://docs.treasuredata.com/articles/hive-functions) for details, (and consider `CONCAT_WS`, for example).

Discuss your findings.

**Answer**

In [78]:
tweets.describe('user.followers_count').show()

+-------+--------------------+
|summary|user.followers_count|
+-------+--------------------+
|  count|              100000|
|   mean|          4989.93252|
| stddev|  127175.56091358254|
|    min|                   0|
|    max|            18937529|
+-------+--------------------+



* **We assumed sets of hashtags of users who had 1.5 times of average number of followers were popular.**
* **We assumed sets of hashtags of users who had 0.5 times of average number of followers were unpopular.**

In [79]:
sqlc.sql("""
SELECT 
AVG(user.followers_count)/2 AS first_quarter,
AVG(user.followers_count) AS mean, 
AVG(user.followers_count)*3/2 AS third_quarter
FROM tweets
""").show()

+-------------+----------+-------------+
|first_quarter|      mean|third_quarter|
+-------------+----------+-------------+
|   2494.96626|4989.93252|   7484.89878|
+-------------+----------+-------------+



In [80]:
tweets.filter(tweets.user.followers_count>7485) \
.groupby('entities.hashtags.text').count().orderBy("count", ascending=False).show(10)

+--------------------+-----+
|                text|count|
+--------------------+-----+
|                  []| 1949|
|       [WorldSeries]|  256|
|       [EarnHistory]|  145|
|            [Astros]|  106|
|           [Dodgers]|   62|
|         [ASTROSWIN]|   43|
|               [MLB]|   30|
|      [SerieMundial]|   23|
|[walkoff, WorldSe...|   22|
|[EarnHistory, Wor...|   22|
+--------------------+-----+
only showing top 10 rows



In [81]:
sqlc.sql("""
SELECT CONCAT_WS(',', entities.hashtags['text']) AS name, COUNT(*) AS count
FROM tweets
WHERE user.followers_count > 7485
GROUP BY name
ORDER BY count DESC
LIMIT 10
""").show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|                    | 1949|
|         WorldSeries|  256|
|         EarnHistory|  145|
|              Astros|  106|
|             Dodgers|   62|
|           ASTROSWIN|   43|
|                 MLB|   30|
|        SerieMundial|   23|
| walkoff,WorldSeries|   22|
|EarnHistory,World...|   22|
+--------------------+-----+



**We got the same result by SQL query and spark python codes.**

In [82]:
tweets.filter(tweets.user.followers_count<2494) \
.groupby('entities.hashtags.text').count().orderBy("count", ascending=False).show(10)

+--------------------+-----+
|                text|count|
+--------------------+-----+
|                  []|50323|
|       [EarnHistory]|10365|
|       [WorldSeries]| 6977|
|         [ASTROSWIN]| 3379|
|            [Astros]| 1642|
|           [Dodgers]| 1013|
|[walkoff, WorldSe...|  611|
|[ThisTeam, WorldS...|  587|
|          [ThisTeam]|  535|
|[Astros, WorldSer...|  414|
+--------------------+-----+
only showing top 10 rows



In [83]:
sqlc.sql("""
SELECT CONCAT_WS(',', entities.hashtags['text']) AS name, COUNT(*) AS count
FROM tweets
WHERE user.followers_count < 2494
GROUP BY name
ORDER BY count DESC
LIMIT 10
""").show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|                    |50323|
|         EarnHistory|10365|
|         WorldSeries| 6977|
|           ASTROSWIN| 3379|
|              Astros| 1642|
|             Dodgers| 1013|
| walkoff,WorldSeries|  611|
|ThisTeam,WorldSeries|  587|
|            ThisTeam|  535|
|Astros,WorldSerie...|  414|
+--------------------+-----+



**We got the same result by SQL query and spark python codes.**

### Q3.5 - Analyze common words in tweet text

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", add a filter that removes at least 10 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  To split lines into words, a simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation.

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, are they more frequently used by Dodgers or Astros fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

In [30]:
wordlist=["an", "the", "in","to","at","this","and","for","is","rt","it","of"]
def process(line,wordlist):
    words=[]
    line=line.lower()
    for word in line.split():
        if len(word)>1 and not(word in wordlist):
            words.append(word)
    return words

**"an","the","in","to","at","this","and","for","is","it","of","rt" are our stop words. In order to ignore the case issue, we lowered all words before counting. Therefore, we used "rt" rather than "RT" in stop word list.**

In [35]:
tweets.rdd.flatMap(lambda r: process(r['text'],wordlist)) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda pair: -pair[1])

[('astros', 31412),
 ('dodgers', 18358),
 ('game', 15963),
 ('@astros:', 15411),
 ('win', 14945),
 ('#earnhistory', 14873),
 ('#worldseries', 13801),
 ('@astros', 10746),
 ('world', 8392),
 ('series', 8141)]

**As the result shown above, these ten words were the top 10 most common words in tweet text. Since we didn't account for punctuation, there are some repeat words due to the punctuation.**

In [37]:
tweets.filter(tweets.user.location == 'Houston, TX').rdd.flatMap(lambda r: process(r['text'],wordlist)) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda pair: -pair[1])

[('#earnhistory', 2036),
 ('astros', 2014),
 ('@astros:', 1999),
 ('win', 1379),
 ('#worldseries', 1015),
 ('game', 1002),
 ('@astros', 813),
 ('#astros', 788),
 ('@abreg_1', 652),
 ('world', 637)]

In [38]:
tweets.filter(tweets.user.location == 'Los Angeles, CA').rdd.flatMap(lambda r: process(r['text'],wordlist)) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda pair: -pair[1])

[('dodgers', 601),
 ('astros', 398),
 ('game', 322),
 ('@dodgers', 299),
 ('#worldseries', 275),
 ('#thisteam', 267),
 ('win', 199),
 ('you', 187),
 ('on', 152),
 ('series', 145)]

**We were wondering if different location had different top common words. Therefore, we counted top common words in tweets with the location of "Los Angeles" and "Houston".**
<br>
**As the results shown above, in Houston, people only talked about thier team Astros. However, in Los Angeles, people not only talked about Dodgers, but also their opponent Astros. Then, people in both locations talked about "world-series", and of course "win".**